# Fitting py3DCORE_h4c

In [ ]:
%load_ext autoreload

%autoreload 2

import heliosat
import logging
import datetime
import numpy as np
import os
import pickle
import py3dcore_h4c as py3dcore_h4c
import matplotlib.pyplot as plt
import shutil
import pandas as pds
import event as evt

In [ ]:
from heliosat.util import sanitize_dt

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")

## ICMECAT

We first read the catalog and save all events in lists.

In [ ]:
wincat,stacat,stbcat,pspcat,solocat,bepicat,ulycat,messcat,vexcat = evt.get_cat()

i1au = wincat + stacat + stbcat
print('ICMECAT events near 1 AU',len(i1au))

We now choose a specific event we want to fit.

In [ ]:
winevent = evt.findevent(wincat, year=2022,month=2,day=3)

print('Start of event: '+str(winevent[0].begin))
print('Start of MC: '+str(winevent[0].cloud))
print('End of event: '+str(winevent[0].end))

We set the launchtime for the CME, t_launch (as observed), and two points in time (t_s and t_e), that lie outside of our fitting range. These function as reference points to check, whether an ensemble is actually hitting our spacecraft within the time window. Furthermore, we set fitting points, t, to which the flux rope model will be fitted.

In [ ]:
#t_launch = winevent[0].begin-datetime.timedelta(days=4)

t_launch = datetime.datetime(2022, 1, 29, 12,tzinfo=datetime.timezone.utc)

t_s_wind = datetime.datetime(2022, 2, 2, 18, tzinfo=datetime.timezone.utc)
t_e_wind = datetime.datetime(2022, 2, 3, 14,tzinfo=datetime.timezone.utc)

t_wind = [
    datetime.datetime(2022, 2, 3,2, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 2, 3,4, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 2, 3,6, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 2, 3,8, tzinfo=datetime.timezone.utc)
]

In [ ]:
winevent[0].begin - t_s_wind

Restraining the initial values for the ensemble members leads to more efficient fitting.

    Model Parameters
    ================
        For this specific model there are a total of 14 initial parameters which are as follows:
        0: t_i          time offset
        1: lon          longitude
        2: lat          latitude
        3: inc          inclination

        4: dia          cross section diameter at 1 AU
        5: delta        cross section aspect ratio

        6: r0           initial cme radius
        7: v0           initial cme velocity
        8: T            T factor (related to the twist)

        9: n_a          expansion rate
        10: n_b         magnetic field decay rate

        11: b           magnetic field strength at center at 1AU
        12: bg_d        solar wind background drag coefficient
        13: bg_v        solar wind background speed

        There are 4 state parameters which are as follows:
        0: v_t          current velocity
        1: rho_0        torus major radius
        2: rho_1        torus minor radius
        3: b_t          magnetic field strength at center

In [ ]:
model_kwargs = {
    "ensemble_size": int(2**18), #2**17
    "iparams": {
       "cme_longitude": {
           "maximum": 10,
           "minimum": -15
       },
       "cme_latitude": {
           "maximum": 15,
           "minimum": -25
       },
       "cme_inclination": {
           "maximum": 90,
           "minimum": 0
       }, 
        "cme_launch_velocity": {
            "maximum": 900,
            "minimum": 500
        },
        "cme_launch_radius": {
            "default_value": 20
        }
    }
}

In [ ]:
#"cme_launch_offset": {
#    "distribution": "uniform",
#    "maximum": 3600*24,
#    "minimum": -3600*24
#}

In [ ]:
#first clean output folder
output="out_wind_heeq/"

# Deleting an non-empty folder
try:
    shutil.rmtree(output, ignore_errors=True)
    print("Deleted '%s' directory successfully" % output)
except:
    pass



In [ ]:
output_test = 'out_wind_heeq_512/'

fitter = py3dcore_h4c.ABC_SMC()
fitter.initialize(t_launch, py3dcore_h4c.ToroidalModel, model_kwargs)
fitter.add_observer("WIND", t_wind, t_s_wind, t_e_wind)

fitter.run(ensemble_size=512, reference_frame="HEEQ", jobs=8, workers=8, sampling_freq=3600, output=output_test) 

now that the fitting is completed, we look at the results.

In [ ]:
# Get the list of all files and directories
path = output_test
dir_list = sorted(os.listdir(path))
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

resultpath = path + dir_list[-1]
resultpath2 = path + dir_list[-2]
print(resultpath)

In [ ]:
t_s_wind = datetime.datetime(2022, 2, 1, 18, tzinfo=datetime.timezone.utc)
t_e_wind = datetime.datetime(2022, 2, 4, 18,tzinfo=datetime.timezone.utc)

wind_t, wind_b = heliosat.WIND().get([t_s_wind, t_e_wind], "mag", reference_frame="HEEQ", as_endpoints=True, return_datetimes=True, smoothing="gaussian")

for dirl in dir_list:
    
    resultpath = path + dirl
    ed = py3dcore_h4c.generate_ensemble(resultpath, wind_t, reference_frame="HEEQ", reference_frame_to="HEEQ", max_index=128)

    plt.figure(figsize=(28, 12))
    plt.title("Wind fitting result")
    plt.plot(wind_t, np.sqrt(np.sum(wind_b**2, axis=1)), "k", alpha=0.5)
    plt.plot(wind_t, wind_b[:, 0], "r", alpha=0.5)
    plt.plot(wind_t, wind_b[:, 1], "g", alpha=0.5)
    plt.plot(wind_t, wind_b[:, 2], "b", alpha=0.5)
    plt.fill_between(wind_t, ed[0][3][0], ed[0][3][1], alpha=0.25, color="k")
    plt.fill_between(wind_t, ed[0][2][0][:, 0], ed[0][2][1][:, 0], alpha=0.25, color="r")
    plt.fill_between(wind_t, ed[0][2][0][:, 1], ed[0][2][1][:, 1], alpha=0.25, color="g")
    plt.fill_between(wind_t, ed[0][2][0][:, 2], ed[0][2][1][:, 2], alpha=0.25, color="b")
    plt.ylabel("B [nT]")
    plt.xlabel("Time [MM-DD HH]")
    for _ in t_wind:
        plt.axvline(x=_, lw=1, alpha=0.25, color="k", ls="--")
    plt.savefig(path+'%s.png' % dirl)
    plt.show()